### Feature Extraction with a Pretrained Model

Implemented with GraphLab Create

In [ ]:
WORKING_DIR = '/home/ubuntu/kaggle/NaiveBees'
import graphlab as gl
import os

In [ ]:
train_dir = WORKING_DIR + '/train'
test_dir = WORKING_DIR + '/test'

In [ ]:
train_sf = gl.image_analysis.load_images(train_dir, random_order=True)

In [ ]:
train_sf['image'] = gl.image_analysis.resize(train_sf['image'], 256, 256)

In [ ]:
train_sf['id'] = train_sf['path'].apply(lambda x: x.split('/')[-1])

In [ ]:
train_sf['id'] = train_sf['id'].apply(lambda x: int(x.split('.')[-2]))

In [ ]:
labels_sf = gl.SFrame.read_csv(WORKING_DIR + '/train.csv')

In [ ]:
train = train_sf.join(labels_sf, on='id')

In [ ]:
#mean_image = train['image'].mean()

In [ ]:
deep_learning_model = gl.load_model(
    'http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')

In [ ]:
train['deep_features'] = deep_learning_model.extract_features(train)

In [ ]:
train['genus'] = train['genus'].apply(lambda x: int(x))

In [ ]:
train_sf, val_sf = train.random_split(0.85)

In [ ]:
deep_features_model = graphlab.logistic_classifier.create(train_sf, features=['deep_features'],target='genus', validation_set=None)

In [ ]:
results = deep_features_model.evaluate(val_sf, metric='roc_curve')

In [ ]:
results['roc_curve']

In [ ]:
graphlab.canvas.set_target('ipynb')
deep_features_model.show(view='Evaluation')

In [ ]:
test_sf = gl.image_analysis.load_images(test_dir, random_order=True)
test_sf['image'] = gl.image_analysis.resize(test_sf['image'], 256, 256)
test_sf['id'] = test_sf['path'].apply(lambda x: x.split('/')[-1])
test_sf['id'] = test_sf['id'].apply(lambda x: int(x.split('.')[-2]))

In [ ]:
test_sf['deep_features'] = deep_learning_model.extract_features(test_sf)

In [ ]:
deep_features_model = graphlab.logistic_classifier.create(train, features=['deep_features'],target='genus', validation_set=None)

In [ ]:
predictions = deep_features_model.predict(test_sf,  output_type='probability')

In [ ]:
predictions

In [ ]:
test = test_sf.add_column(predictions, name='genus')

In [ ]:
submission_format = gl.SFrame.read_csv(WORKING_DIR + '/SubmissionFormat.csv')

In [ ]:
submission = submission_format['id'].join(test['id','genus'], on={'id':'id'}, how='inner')

In [ ]:
submission

In [ ]:
submission.export_csv(WORKING_DIR+'/submission_deepfeatures_logisticregression.csv')